In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [61]:
transac = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx',sheet_name = 'Transactions',skiprows=1)
new_cust = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx',sheet_name = 'NewCustomerList',skiprows=1)
cust_demog = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx',sheet_name = 'CustomerDemographic',skiprows=1)
cust_addrs = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx',sheet_name = 'CustomerAddress',skiprows=1)

In [62]:
new_cust = new_cust.drop(columns = ['first_name','last_name','deceased_indicator','Rank','Value','country'])


In [63]:
transac['Margin'] = transac.list_price - transac.standard_cost
customer_margin = dict(transac.groupby(['customer_id']).sum().Margin)

Joining old customer data

In [64]:
cust_demo_addr = cust_addrs.merge(cust_demog,left_on='customer_id',right_on='customer_id',)

In [65]:
cust_demo_addr = cust_demo_addr.drop(columns = ['first_name','last_name','country','address','default','deceased_indicator'])

In [66]:
cust_demo_addr['Margin'] = cust_demo_addr.customer_id.apply(lambda x : customer_margin[x] if x in customer_margin.keys() else 0)

In [67]:

pd.options.mode.chained_assignment = None  # default='warn'

cust_transctd = transac.customer_id.unique()

label = cust_demo_addr.customer_id.apply(lambda x : 1 if x in cust_transctd else 0).values

cust_demo_addr['Target'] = label
cust_demo_addr['set'] = 'train'

cust_demo_addr = cust_demo_addr.iloc[:,1:]
cust_demo_addr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3996 entries, 0 to 3995
Data columns (total 14 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   postcode                             3996 non-null   int64         
 1   state                                3996 non-null   object        
 2   property_valuation                   3996 non-null   int64         
 3   gender                               3996 non-null   object        
 4   past_3_years_bike_related_purchases  3996 non-null   int64         
 5   DOB                                  3909 non-null   datetime64[ns]
 6   job_title                            3492 non-null   object        
 7   job_industry_category                3341 non-null   object        
 8   wealth_segment                       3996 non-null   object        
 9   owns_car                             3996 non-null   object        
 10  tenure      

In [72]:
cust_demo_addr.Margin.to_csv('margin.csv')

In [10]:
new_cust = new_cust.iloc[:,:12]
new_cust = new_cust[['postcode', 'property_valuation', 'past_3_years_bike_related_purchases',
       'owns_car', 'tenure', 'DOB', 'gender', 'state',
       'job_title','job_industry_category', 'wealth_segment']]
new_cust['set'] = 'test'
new_cust['Target'] =  0
new_cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   postcode                             1000 non-null   int64         
 1   property_valuation                   1000 non-null   int64         
 2   past_3_years_bike_related_purchases  1000 non-null   int64         
 3   owns_car                             1000 non-null   object        
 4   tenure                               1000 non-null   int64         
 5   DOB                                  983 non-null    datetime64[ns]
 6   gender                               1000 non-null   object        
 7   state                                1000 non-null   object        
 8   job_title                            894 non-null    object        
 9   job_industry_category                835 non-null    object        
 10  wealth_segmen

In [11]:
cust_demo_addr = pd.concat([cust_demo_addr,new_cust])
cust_demo_addr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4996 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   postcode                             4996 non-null   int64         
 1   state                                4996 non-null   object        
 2   property_valuation                   4996 non-null   int64         
 3   gender                               4996 non-null   object        
 4   past_3_years_bike_related_purchases  4996 non-null   int64         
 5   DOB                                  4892 non-null   datetime64[ns]
 6   job_title                            4386 non-null   object        
 7   job_industry_category                4176 non-null   object        
 8   wealth_segment                       4996 non-null   object        
 9   owns_car                             4996 non-null   object        
 10  tenure       

Creating Age column and mitigaing gender column inconsistencies where category 'U' is set to Null and imputed later

In [12]:
cust_demo_addr['Age'] = 2020 - cust_demo_addr.DOB.dt.year
cust_demo_addr.loc[cust_demo_addr.Age == cust_demo_addr['Age'].max(),'Age'] = 80
cust_demo_addr.gender = cust_demo_addr.gender.apply(lambda x: x[0].upper() if x != 'U' else np.nan)
del cust_demo_addr['DOB']

Label encoding

In [13]:
cust_demo_addr.owns_car = cust_demo_addr.owns_car.apply(lambda x: 0 if x == 'No' else 1)

Mitigating state values inconsistencies

In [14]:

def state_replace(x):
    if x == 'New South Wales':
        x = 'NSW'
    if x == 'Victoria':
        x = 'VIC'
    return x

cust_demo_addr.state = cust_demo_addr.state.apply(lambda x: state_replace(x))


In [15]:
cust_demo_addr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4996 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   postcode                             4996 non-null   int64  
 1   state                                4996 non-null   object 
 2   property_valuation                   4996 non-null   int64  
 3   gender                               4891 non-null   object 
 4   past_3_years_bike_related_purchases  4996 non-null   int64  
 5   job_title                            4386 non-null   object 
 6   job_industry_category                4176 non-null   object 
 7   wealth_segment                       4996 non-null   object 
 8   owns_car                             4996 non-null   int64  
 9   tenure                               4909 non-null   float64
 10  Margin                               3996 non-null   float64
 11  Target                         

Mean Imputing Age and Tenure by grouping on job title and job industry category

In [16]:
job_group = cust_demo_addr.groupby(['job_title','job_industry_category']).mean()
job_group = job_group.fillna(job_group.mean())
job_group

postcode  property_valuation  \
job_title           job_industry_category                                 
Account Coordinator Argiculture              2871.0                3.00   
                    Entertainment            2147.0                8.00   
                    Financial Services       2388.0                5.00   
                    Health                   3352.0                5.75   
                    IT                       3631.5                7.50   
...                                             ...                 ...   
Web Developer IV    Argiculture              2759.0                9.00   
                    Entertainment            2540.0                9.00   
                    Manufacturing            3217.0                8.50   
                    Property                 4077.0                5.00   
                    Retail                   2168.0                9.00   

                                           past_3_years_bike_related_purchases  \
job_title           job_industry_category                                        
Account Coordinator Argiculture                                          71.00   
                    Entertainment                                        33.00   
                    Financial Services                                   18.00   
                    Health                                               25.75   
                    IT                                                   28.00   
...                                                                        ...   
Web Developer IV    Argiculture                                           5.00   
                    Entertainment                                        89.00   
                    Manufacturing                                        37.50   
                    Property                                             44.00   
                    Retail                                               87.00   

                                           owns_car  tenure       Margin  \
job_title           job_industry_category                                  
Account Coordinator Argiculture                 0.0   17.00  2738.890000   
                    Entertainment               1.0    3.00     0.000000   
                    Financial Services          1.0   14.00  3408.455000   
                    Health                      0.0   12.25  1529.146667   
                    IT                          0.0   15.00  3753.740000   
...                                             ...     ...          ...   
Web Developer IV    Argiculture                 1.0   21.00  5580.570000   
                    Entertainment               1.0   15.00  7335.890000   
                    Manufacturing               0.5   11.00     0.000000   
                    Property                    0.0    4.00     0.000000   
                    Retail                      0.0   12.00  1785.830000   

                                           Target   Age  
job_title           job_industry_category                
Account Coordinator Argiculture               1.0  43.0  
                    Entertainment             0.0  42.0  
                    Financial Services        1.0  44.0  
                    Health                    0.5  51.0  
                    IT                        1.0  64.0  
...                                           ...   ...  
Web Developer IV    Argiculture               1.0  46.0  
                    Entertainment             1.0  48.0  
                    Manufacturing             0.0  34.5  
                    Property                  0.0  22.0  
                    Retail                    1.0  40.0  

[976 rows x 8 columns]

Accessing group df multiple index to fetch valuess. If Job title is not found takes mean of the jon category.

In [17]:
age_imputed = []
tenure_imputed = []
for age,tenure,index in zip(cust_demo_addr.Age,cust_demo_addr.tenure,cust_demo_addr.index):
    if pd.isnull(age):
        job_det = cust_demo_addr.iloc[index,:]
        job_title = job_det['job_title']
        job_industry_category = job_det['job_industry_category']
        
        if pd.isnull(job_industry_category) != True:
            try:    
                age_imputed.append(int(job_group.loc[(job_title,job_industry_category)].Age))
                tenure_imputed.append(int(job_group.loc[(job_title,job_industry_category)].tenure))
            except:
                age_imputed.append(int(job_group.xs(job_industry_category, level='job_industry_category').Age.mean()))
                tenure_imputed.append(int(job_group.xs(job_industry_category, level='job_industry_category').tenure.mean()))
        else:
            age_imputed.append(np.nan)
            tenure_imputed.append(np.nan)
    else:
        age_imputed.append(age)
        tenure_imputed.append(tenure)

In [18]:
cust_demo_addr.Age = age_imputed
cust_demo_addr.tenure = tenure_imputed
cust_demo_addr.Age = cust_demo_addr.Age.fillna(cust_demo_addr.Age.mean())

In [19]:
cust_demo_addr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4996 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   postcode                             4996 non-null   int64  
 1   state                                4996 non-null   object 
 2   property_valuation                   4996 non-null   int64  
 3   gender                               4891 non-null   object 
 4   past_3_years_bike_related_purchases  4996 non-null   int64  
 5   job_title                            4386 non-null   object 
 6   job_industry_category                4176 non-null   object 
 7   wealth_segment                       4996 non-null   object 
 8   owns_car                             4996 non-null   int64  
 9   tenure                               4993 non-null   float64
 10  Margin                               3996 non-null   float64
 11  Target                         

Imputing Job Category info

Data structure = { Category : [ Titles], ....}

In [20]:
job_det = {}
for category in cust_demo_addr.job_industry_category.dropna().unique():
    job_titles = cust_demo_addr[cust_demo_addr.job_industry_category==category].job_title
    job_det[category] = list(job_titles.dropna().unique())

job_det['Health']

['Executive Secretary',
 'Research Nurse',
 'Senior Quality Engineer',
 'Information Systems Manager',
 'Social Worker',
 'Food Chemist',
 'Registered Nurse',
 'Occupational Therapist',
 'Speech Pathologist',
 'Recruiting Manager',
 'Safety Technician IV',
 'Automation Specialist II',
 'Software Test Engineer III',
 'Account Executive',
 'Administrative Assistant II',
 'Administrative Officer',
 'Quality Engineer',
 'Analog Circuit Design manager',
 'Pharmacist',
 'General Manager',
 'Junior Executive',
 'Database Administrator III',
 'Clinical Specialist',
 'Legal Assistant',
 'Director of Sales',
 'Senior Developer',
 'Dental Hygienist',
 'Technical Writer',
 'VP Sales',
 'Internal Auditor',
 'Design Engineer',
 'Desktop Support Technician',
 'Human Resources Manager',
 'Programmer IV',
 'Senior Sales Associate',
 'Software Engineer IV',
 'Structural Analysis Engineer',
 'Project Manager',
 'Programmer Analyst II',
 'Structural Engineer',
 'Help Desk Operator',
 'Sales Associate',
 '

Imputing places where Job category is not present and job title is present.

In [21]:
job_cat_nan = cust_demo_addr[~cust_demo_addr.job_title.isna() & cust_demo_addr.job_industry_category.isna()].job_title

impute_category = []

for job_titl in job_cat_nan:
    for key,value in job_det.items():
        if job_titl in value:
            impute_category.append(key)
            break
            

In [22]:
imputed_idx = cust_demo_addr[~cust_demo_addr.job_title.isna() & cust_demo_addr.job_industry_category.isna()].job_industry_category.index

cust_demo_addr.iloc[imputed_idx,6] = impute_category

In [23]:
cust_demo_addr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4996 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   postcode                             4996 non-null   int64  
 1   state                                4996 non-null   object 
 2   property_valuation                   4996 non-null   int64  
 3   gender                               4891 non-null   object 
 4   past_3_years_bike_related_purchases  4996 non-null   int64  
 5   job_title                            4386 non-null   object 
 6   job_industry_category                4732 non-null   object 
 7   wealth_segment                       4996 non-null   object 
 8   owns_car                             4996 non-null   int64  
 9   tenure                               4993 non-null   float64
 10  Margin                               3996 non-null   float64
 11  Target                         

KNN imputation for the missing values in Gender & Remmaining Job related fields

In [24]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder


In [25]:
imputer = KNNImputer(n_neighbors=7)

In [26]:
cust_demo_addr_imputed = cust_demo_addr.copy()
cust_demo_addr_imputed = pd.DataFrame(imputer.fit_transform(pd.get_dummies(cust_demo_addr)))

On ehot encoding all columns correspondingly

In [27]:
cust_demo_addr_imputed.columns = pd.get_dummies(cust_demo_addr).columns
cust_demo_addr_imputed

,postcode,property_valuation,past_3_years_bike_related_purchases,owns_car,tenure,Margin,Target,Age,state_NSW,state_QLD,...,job_industry_category_IT,job_industry_category_Manufacturing,job_industry_category_Property,job_industry_category_Retail,job_industry_category_Telecommunications,wealth_segment_Affluent Customer,wealth_segment_High Net Worth,wealth_segment_Mass Customer,set_test,set_train
0,2016.0,10.0,93.0,1.0,11.0,3018.090000,1.0,67.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,2153.0,10.0,81.0,1.0,16.0,2226.260000,1.0,40.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,4211.0,9.0,33.0,0.0,7.0,220.570000,1.0,59.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,2448.0,4.0,56.0,1.0,8.0,2394.940000,1.0,43.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,3216.0,9.0,35.0,1.0,13.0,3946.550000,1.0,54.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4991,2200.0,7.0,60.0,0.0,9.0,2717.242857,0.0,61.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4992,2196.0,10.0,22.0,0.0,6.0,2071.628571,0.0,19.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4993,4702.0,2.0,17.0,1.0,15.0,2939.068571,0.0,66.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4994,4215.0,2.0,30.0,1.0,19.0,3476.851429,0.0,68.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


Reversing one hot encoing using idmax for each set of columns

In [28]:
wide_to_long_prefx = ['gender','state','job_title','job_industry_category','wealth_segment','set']

for col in wide_to_long_prefx:
    
    print(col)
    cust_demo_addr_imputed [col] = cust_demo_addr_imputed.filter(like=col, axis=1).idxmax(1).apply(lambda x: x.replace(col+'_',''))
    
    for cols in cust_demo_addr_imputed.filter(like=col+'_', axis=1).columns:
        del cust_demo_addr_imputed[cols]

gender
state
job_title
job_industry_category
wealth_segment
set


In [29]:
cust_demo_addr_imputed = cust_demo_addr_imputed [['postcode', 'property_valuation', 'past_3_years_bike_related_purchases',
       'owns_car', 'tenure', 'Age', 'gender', 'state',
       'job_title','job_industry_category', 'wealth_segment','Target','set']]

cust_demo_addr_imputed['Age'] = cust_demo_addr_imputed.Age.apply(lambda x: 'Young' if x <30 else ( 'Adult' if x < 50 else 'elderly'))
cust_demo_addr_imputed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4996 entries, 0 to 4995
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   postcode                             4996 non-null   float64
 1   property_valuation                   4996 non-null   float64
 2   past_3_years_bike_related_purchases  4996 non-null   float64
 3   owns_car                             4996 non-null   float64
 4   tenure                               4996 non-null   float64
 5   Age                                  4996 non-null   object 
 6   gender                               4996 non-null   object 
 7   state                                4996 non-null   object 
 8   job_title                            4996 non-null   object 
 9   job_industry_category                4996 non-null   object 
 10  wealth_segment                       4996 non-null   object 
 11  Target                        

In [30]:
train = cust_demo_addr_imputed[cust_demo_addr_imputed.set == 'train']
test = cust_demo_addr_imputed[cust_demo_addr_imputed.set == 'test']

del train['set']

del test['set']

del train['job_title']

del test['job_title']

train_traget = train['Target']
del train['Target']

del test['Target']

In [31]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)
train

,postcode,property_valuation,past_3_years_bike_related_purchases,owns_car,tenure,Age_Adult,Age_Young,Age_elderly,gender_F,gender_M,...,job_industry_category_Financial Services,job_industry_category_Health,job_industry_category_IT,job_industry_category_Manufacturing,job_industry_category_Property,job_industry_category_Retail,job_industry_category_Telecommunications,wealth_segment_Affluent Customer,wealth_segment_High Net Worth,wealth_segment_Mass Customer
0,2016.0,10.0,93.0,1.0,11.0,0,0,1,1,0,...,0,1,0,0,0,0,0,0,0,1
1,2153.0,10.0,81.0,1.0,16.0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,1
2,4211.0,9.0,33.0,0.0,7.0,0,0,1,0,1,...,0,0,1,0,0,0,0,0,0,1
3,2448.0,4.0,56.0,1.0,8.0,1,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
4,3216.0,9.0,35.0,1.0,13.0,0,0,1,0,1,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3991,3977.0,6.0,8.0,0.0,19.0,1,0,0,1,0,...,0,1,0,0,0,0,0,0,0,1
3992,2350.0,2.0,87.0,1.0,1.0,0,1,0,1,0,...,0,0,0,1,0,0,0,0,1,0
3993,2540.0,6.0,60.0,0.0,10.0,1,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0
3994,3064.0,3.0,11.0,1.0,10.0,1,0,0,0,1,...,0,0,0,1,0,0,0,1,0,0


In [32]:
multicollinear_mitigate_cols = ['postcode', 'property_valuation', 'past_3_years_bike_related_purchases',
       'owns_car', 'tenure', 'Age_Young','Age_elderly', 'gender_F', 'state_NSW',
       'state_QLD', 'job_industry_category_Argiculture',
       'job_industry_category_Entertainment',
       'job_industry_category_Financial Services',
       'job_industry_category_Health', 'job_industry_category_IT',
       'job_industry_category_Manufacturing', 'job_industry_category_Property',
       'job_industry_category_Retail',
       'wealth_segment_Affluent Customer', 'wealth_segment_High Net Worth']

In [33]:
train = train[multicollinear_mitigate_cols]
test = test [multicollinear_mitigate_cols]

In [34]:
print('Actual Minotity class ratio = {}'.format(507/3489))

Actual Minotity class ratio = 0.14531384350816853


In [35]:
from imblearn.over_sampling import SMOTE

def upsample_SMOTE(X_train, y_train, ratio=0.7):
    """Upsamples minority class using SMOTE.
    Ratio argument is the percentage of the upsampled minority class in relation
    to the majority class. Default is 1.0
    """
    sm = SMOTE(random_state=23, sampling_strategy=ratio)
    X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)
    print(len(X_train_sm), len(y_train_sm))
    return X_train_sm, y_train_sm

Using TensorFlow backend.


In [36]:
X_train_sm, y_train_sm = upsample_SMOTE(train.values, train_traget.values)

5931 5931


In [37]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_train_sm, y_train_sm, test_size = 0.25, random_state = 0)


In [38]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0,max_features=5)
classifier.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=5, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [39]:
from sklearn.metrics import confusion_matrix
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
cm

array([[494, 123],
       [129, 737]], dtype=int64)

In [40]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names=['Not Targeted','Targeted']))

              precision    recall  f1-score   support

Not Targeted       0.79      0.80      0.80       617
    Targeted       0.86      0.85      0.85       866

    accuracy                           0.83      1483
   macro avg       0.82      0.83      0.83      1483
weighted avg       0.83      0.83      0.83      1483



In [41]:
new_custoer_target = classifier.predict(test.values)

print('New customer targeted count')
np.array(np.unique(new_custoer_target, return_counts=True)).T

New customer targeted count


array([[  0., 147.],
       [  1., 853.]])

In [42]:
144/855

0.16842105263157894

In [43]:
# Visualize data
import pydotplus
from sklearn import tree
import collections
import os

os.environ['PATH'] = os.environ['PATH']+';'+os.environ['CONDA_PREFIX']+r"\Library\bin\graphviz"

dot_data = tree.export_graphviz(classifier,
                                feature_names=multicollinear_mitigate_cols,
                                out_file=None,
                                filled=True,
                                rounded=True)
graph = pydotplus.graph_from_dot_data(dot_data)

colors = ('turquoise', 'orange')
edges = collections.defaultdict(list)

for edge in graph.get_edge_list():
    edges[edge.get_source()].append(int(edge.get_destination()))

for edge in edges:
    edges[edge].sort()    
    for i in range(2):
        dest = graph.get_node(str(edges[edge][i]))[0]
        dest.set_fillcolor(colors[i])

graph.write_png('tree.png')

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.611815 to fit



True

## Feature eng & Interpretation

No significant correlation between continuous features.

In [44]:
cust_demo_addr_imputed.corr()

,postcode,property_valuation,past_3_years_bike_related_purchases,owns_car,tenure,Target
postcode,1.000000,-0.506293,-0.002847,-0.001923,0.004035,-0.005581
property_valuation,-0.506293,1.000000,0.005124,0.016891,-0.008175,0.013698
past_3_years_bike_related_purchases,-0.002847,0.005124,1.000000,-0.009078,-0.014656,-0.014722
owns_car,-0.001923,0.016891,-0.009078,1.000000,-0.004370,0.009879
tenure,0.004035,-0.008175,-0.014656,-0.004370,1.000000,-0.036219
Target,-0.005581,0.013698,-0.014722,0.009879,-0.036219,1.000000


Chi Square test

    -Job Industry category has association with Target variable (p value < 0.05)
    -Job Title does not have any association with target variable (pvalue > 0.05)

In [45]:
from scipy.stats import chi2_contingency

target_job_title_cross_tab = pd.crosstab(cust_demo_addr.Target.values,cust_demo_addr.job_industry_category.values).values

chi2_contingency(target_job_title_cross_tab)[:2]


(35.141878318376435, 2.519325674953917e-05)

In [46]:
target_job_title_cross_tab = pd.crosstab(cust_demo_addr.Target.values,cust_demo_addr.job_title.values).values

chi2_contingency(target_job_title_cross_tab)[:2]

(168.01858386806512, 0.9112864005831538)

In [47]:
import statsmodels.api as sm

X = train.iloc[:,:7]
y = train_traget.values

X.columns

Index(['postcode', 'property_valuation', 'past_3_years_bike_related_purchases',
       'owns_car', 'tenure', 'Age_Young', 'Age_elderly'],
      dtype='object')

In [48]:
logit = sm.Logit(y, X[['postcode', 'property_valuation', 'tenure']])
fit = logit.fit()

fit.summary()

Optimization terminated successfully.
         Current function value: 0.383583
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                 3996
Model:                          Logit   Df Residuals:                     3993
Method:                           MLE   Df Model:                            2
Date:                Mon, 08 Jun 2020   Pseudo R-squ.:               -0.008351
Time:                        15:51:33   Log-Likelihood:                -1532.8
converged:                       True   LL-Null:                       -1520.1
Covariance Type:            nonrobust   LLR p-value:                     1.000
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
postcode               0.0004   3.42e-05     10.428      0.000       0.000       0.000
property_valuation     0.0866      0.012      7.234      0.000       0.063       0.110
tenure                 0.0196      0.008      2.427      0.015       0.004       0.035
======================================================================================
"""

In [49]:
cust_demo_addr_imputed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4996 entries, 0 to 4995
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   postcode                             4996 non-null   float64
 1   property_valuation                   4996 non-null   float64
 2   past_3_years_bike_related_purchases  4996 non-null   float64
 3   owns_car                             4996 non-null   float64
 4   tenure                               4996 non-null   float64
 5   Age                                  4996 non-null   object 
 6   gender                               4996 non-null   object 
 7   state                                4996 non-null   object 
 8   job_title                            4996 non-null   object 
 9   job_industry_category                4996 non-null   object 
 10  wealth_segment                       4996 non-null   object 
 11  Target                        

In [50]:
cust_demo_addr_imputed[cust_demo_addr_imputed.set == 'train']

,postcode,property_valuation,past_3_years_bike_related_purchases,owns_car,tenure,Age,gender,state,job_title,job_industry_category,wealth_segment,Target,set
0,2016.0,10.0,93.0,1.0,11.0,elderly,F,NSW,Executive Secretary,Health,Mass Customer,1.0,train
1,2153.0,10.0,81.0,1.0,16.0,Adult,M,NSW,Administrative Officer,Financial Services,Mass Customer,1.0,train
2,4211.0,9.0,33.0,0.0,7.0,elderly,M,QLD,Account Coordinator,IT,Mass Customer,1.0,train
3,2448.0,4.0,56.0,1.0,8.0,Adult,F,NSW,Senior Editor,Financial Services,Affluent Customer,1.0,train
4,3216.0,9.0,35.0,1.0,13.0,elderly,M,VIC,Account Coordinator,Retail,High Net Worth,1.0,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3991,3977.0,6.0,8.0,0.0,19.0,Adult,F,VIC,VP Product Management,Health,Mass Customer,0.0,train
3992,2350.0,2.0,87.0,1.0,1.0,Young,F,NSW,Statistician II,Manufacturing,High Net Worth,0.0,train
3993,2540.0,6.0,60.0,0.0,10.0,Adult,F,NSW,Assistant Manager,IT,High Net Worth,0.0,train
3994,3064.0,3.0,11.0,1.0,10.0,Adult,M,VIC,Account Coordinator,Manufacturing,Affluent Customer,0.0,train


In [51]:
train.postcode.value_counts()

2170.0    31
2155.0    30
2145.0    30
2153.0    29
2560.0    26
          ..
4610.0     1
3321.0     1
2335.0     1
2400.0     1
4615.0     1
Name: postcode, Length: 873, dtype: int64

In [52]:
cust_sector_total = dict(cust_demo_addr_imputed.job_industry_category.value_counts())

In [53]:
sector_target_total = dict(cust_demo_addr_imputed.groupby(['job_industry_category']).sum().Target)

In [54]:
op = {}
for key in cust_sector_total.keys():
    op[key] = int((sector_target_total[key] / cust_sector_total[key])*100)
    
op_df = pd.DataFrame(pd.Series(op).sort_values(ascending = False))
op_df.iloc[:,0] = op_df.iloc[:,0].apply(lambda x:str(x)+'%')
op_df.columns = ['Percentage of sector total']

In [55]:
op_df

,Percentage of sector total
Health,78%
Entertainment,70%
Financial Services,70%
IT,69%
Property,69%
Retail,69%
Manufacturing,69%
Telecommunications,64%
Argiculture,44%


In [56]:
new_custoer_target

array([1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
       1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0.,
       0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 0., 1., 1., 1.

In [57]:
new_cust_target_data = cust_demo_addr_imputed[cust_demo_addr_imputed.set == 'test']
new_cust_target_data.Target = new_custoer_target
new_cust_target_data

,postcode,property_valuation,past_3_years_bike_related_purchases,owns_car,tenure,Age,gender,state,job_title,job_industry_category,wealth_segment,Target,set
3996,4500.0,6.0,86.0,1.0,14.0,elderly,M,QLD,General Manager,Manufacturing,Mass Customer,1.0,test
3997,2113.0,11.0,69.0,0.0,16.0,elderly,M,NSW,Structural Engineer,Property,Mass Customer,1.0,test
3998,3505.0,5.0,10.0,0.0,10.0,Adult,F,VIC,Senior Cost Accountant,Financial Services,Affluent Customer,1.0,test
3999,4814.0,1.0,64.0,1.0,5.0,Adult,F,QLD,Account Representative III,Manufacturing,Affluent Customer,0.0,test
4000,2093.0,9.0,34.0,0.0,19.0,elderly,F,NSW,Financial Analyst,Financial Services,Affluent Customer,1.0,test
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4991,2200.0,7.0,60.0,0.0,9.0,elderly,M,NSW,Paralegal,Financial Services,Affluent Customer,1.0,test
4992,2196.0,10.0,22.0,0.0,6.0,Young,M,NSW,Senior Sales Associate,Health,Mass Customer,1.0,test
4993,4702.0,2.0,17.0,1.0,15.0,elderly,F,QLD,Budget/Accounting Analyst IV,Financial Services,Affluent Customer,0.0,test
4994,4215.0,2.0,30.0,1.0,19.0,elderly,M,QLD,Financial Advisor,Financial Services,Mass Customer,1.0,test


In [58]:
new_cust_target_data.to_csv('target.csv')

PermissionError: [Errno 13] Permission denied: 'target.csv'

In [ ]:
transac.transaction_date.max()

In [ ]:
customer_margin = dict(transac.groupby(['customer_id']).sum().Margin)